In [ ]:

1. zoo classification
2. iris_softmax

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [4]:
path2='C:/Users/김경한/jupyter notebook data/실습데이터/실습데이터/data-04-zoo.csv'

In [8]:
zoo=pd.read_csv(path2,header=None)[19:]
zoo.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
19,1,0,0.0,1.0,0.0,0,1.0,1.0,1.0,1.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0
20,1,0,0.0,1.0,0.0,0,0.0,1.0,1.0,1.0,0.0,0.0,4.0,1.0,0.0,1.0,0.0
21,0,0,1.0,0.0,0.0,1,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,3.0
22,1,0,0.0,1.0,0.0,0,1.0,1.0,1.0,1.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0
23,1,0,0.0,1.0,0.0,0,1.0,1.0,1.0,1.0,0.0,0.0,4.0,1.0,0.0,1.0,0.0


In [11]:
zoo.index=range(len(zoo)) # zoo의 index가 맞지 않으니 전체 길이의 range로 맞춰줌

In [21]:
print(zoo.shape)
Xdata=np.array(zoo.iloc[:,:-1]) # 첫번째 열 부터 맨 뒤 열 까지, train data를 np array로 저장
Ydata=np.array(zoo.iloc[:,-1]).reshape(-1,1) # 마지막 열을 np.array로 바꾸어 reshape

(101, 17)


In [22]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder() # OneHotEncoding 객체 만들기
Ydata = ohe.fit_transform(Ydata).toarray() # Ydata 원핫인코딩

In [23]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest=train_test_split(Xdata,Ydata) # Xdata,Ydata로 train,test나누기

In [25]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler() # scaling을 위한 객체 생성
Xtrain_scaled = scaler.fit_transform(Xtrain)
Xtest_scaled = scaler.transform(Xtest)

In [26]:
x=tf.placeholder(tf.float32,[None,16]) # feature가 16개 이고 많은 행이 입력될 x 생성
y=tf.placeholder(tf.float32,[None,7])

In [27]:
#Weight는 input이 16 output이 7이 되도록 지정
#최종 output이 7
w=tf.Variable(tf.random_normal([16,7],dtype = tf.float32,seed=3))
b=tf.Variable(tf.random_normal([7],dtype = tf.float32,seed=3))

In [29]:
#예측식과 cost함수, 예측에는 softmax함수 사용
hf = tf.nn.softmax(tf.matmul(x,w)+b)
cost = -tf.reduce_sum(y*tf.log(hf),axis=1)

In [30]:
#learning rate=0.1로 cost를 최소화 하도록 model을 training
lr = 0.01
train = tf.train.GradientDescentOptimizer(lr).minimize(cost)

In [34]:
#argmax함수를 이용한 정확도 정의
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(hf,
                                          axis=1),
                                tf.argmax(y,
                                          axis=1)),dtype=tf.float32))
# argmax 함수는 값중 가장 큰 값의 index를 출력할때 사용

In [35]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) #<- 변수를 사용하기 위해 initialize하기
    for step in range(2001):
        sess.run(train, {x:Xtrain_scaled,y:Ytrain}) #x,y에 변수 대입해 train node를 실행
    accVal = sess.run(accuracy, {x:Xtest_scaled,y:Ytest}) #x,y에 변수 대입해 accuracy 실행
    print(accVal)

0.9230769


In [36]:
# keras로 해보기
from keras.models import Sequential
from keras.layers import Dense, Activation

Using TensorFlow backend.


In [38]:
model=Sequential() #Sequetial 객체생성
model.add(Dense(units=64, input_dim=16,activation='relu')) #model에 layer추가
model.add(Dense(units=7,activation = 'softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [39]:
#model을 train하여 결과 나타내기 batch_size:512, epochs:2000
model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])
hist = model.fit(Xtrain_scaled, Ytrain, epochs = 2000, batch_size = 512)
model.evaluate(Xtest_scaled, Ytest, batch_size = 512)[1]



Epoch 1/2000
75/75 [==============================] - 0s 397us/step - loss: 2.0063 - accuracy: 0.0933
Epoch 2/2000
75/75 [==============================] - 0s 13us/step - loss: 1.9840 - accuracy: 0.0933
Epoch 3/2000
75/75 [==============================] - 0s 0us/step - loss: 1.9626 - accuracy: 0.1067
Epoch 4/2000
75/75 [==============================] - 0s 13us/step - loss: 1.9419 - accuracy: 0.1200
Epoch 5/2000
75/75 [==============================] - 0s 13us/step - loss: 1.9222 - accuracy: 0.1200
Epoch 6/2000
75/75 [==============================] - 0s 13us/step - loss: 1.9030 - accuracy: 0.1333
Epoch 7/2000
75/75 [==============================] - 0s 13us/step - loss: 1.8846 - accuracy: 0.2267
Epoch 8/2000
75/75 [==============================] - 0s 22us/step - loss: 1.8667 - accuracy: 0.2533
Epoch 9/2000
75/75 [==============================] - 0s 13us/step - loss: 1.8494 - accuracy: 0.3200
Epoch 10/2000
75/75 [==============================] - 0s 27us/step - loss: 1.8327 - accu

0.8846153616905212

In [ ]:
0.8846153616905212

In [2]:
path='C:/Users/김경한/jupyter notebook data/실습데이터/실습데이터/iris_softmax.csv'

In [3]:
iris_softmax=pd.read_csv(path,header=None)
iris_softmax

,0,1,2,3,4,5,6,7
0,1.0,5.1,3.5,1.4,0.2,1.0,0.0,0.0
1,1.0,4.9,3.0,1.4,0.2,1.0,0.0,0.0
2,1.0,4.7,3.2,1.3,0.2,1.0,0.0,0.0
3,1.0,4.6,3.1,1.5,0.2,1.0,0.0,0.0
4,1.0,5.0,3.6,1.4,0.2,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...
145,1.0,6.7,3.0,5.2,2.3,0.0,0.0,1.0
146,1.0,6.3,2.5,5.0,1.9,0.0,0.0,1.0
147,1.0,6.5,3.0,5.2,2.0,0.0,0.0,1.0
148,1.0,6.2,3.4,5.4,2.3,0.0,0.0,1.0


In [ ]:
'C:/Users/김경한/jupyter notebook data/실습데이터/실습데이터/data-04-zoo.csv'